# BEE 4750 Homework 5: Mixed Integer and Stochastic Programming

**Name**:

**ID**:

> **Due Date**
>
> Thursday, 12/05/24, 9:00pm

## Overview

### Instructions

-   In Problem 1, you will use mixed integer programming to solve a
    waste load allocation problem.

### Load Environment

The following code loads the environment and makes sure all needed
packages are installed. This should be at the start of most Julia
scripts.

In [1]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

  Activating project at `c:\Users\helen\Documents\hw5-newgroup`
┌ Warning: The active manifest file has dependencies that were resolved with a different julia version (1.10.4). Unexpected behavior may occur.
└ @ nothing C:\Users\helen\Documents\hw5-newgroup\Manifest.toml:0


In [3]:
using JuMP
using HiGHS
using DataFrames
using GraphRecipes
using Plots
using Measures
using MarkdownTables

In [ ]:
W = 

## Problems (Total: 30 Points)

### Problem 1 (30 points)

Three cities are developing a coordinated municipal solid waste (MSW)
disposal plan. Three disposal alternatives are being considered: a
landfill (LF), a materials recycling facility (MRF), and a
waste-to-energy facility (WTE). The capacities of these facilities and
the fees for operation and disposal are provided below.

-   **LF**: Capacity 200 Mg, fixed cost \$2000/day, tipping cost
    \$50/Mg;
-   **MRF**: Capacity 350 Mg, fixed cost \$1500/day, tipping cost
    \$7/Mg, recycling cost \$40/Mg recycled;
-   **WTE**: Capacity 210 Mg, fixed cost \$2500/day, tipping cost
    \$60/Mg;

The MRF recycling rate is 40%, and the ash fraction of non-recycled
waste is 16% and of recycled waste is 14%. Transportation costs are
\$1.5/Mg-km, and the relative distances between the cities and
facilities are provided in the table below.

| **City/Facility** | **Landfill (km)** | **MRF (km)** | **WTE (km)** |
|:-----------------:|:-----------------:|:------------:|:------------:|
|         1         |         5         |      30      |      15      |
|         2         |        15         |      25      |      10      |
|         3         |        13         |      45      |      20      |
|        LF         |        \-         |      32      |      18      |
|        MRF        |        32         |      \-      |      15      |
|        WTE        |        18         |      15      |      \-      |

The fixed costs associated with the disposal options are incurred only
if the particular disposal option is implemented. The three cities
produce 100, 90, and 120 Mg/day of solid waste, respectively, with the
composition provided in the table below.

**Reminder**: Use `round(x; digits=n)` to report values to the
appropriate precision!

**In this problem**:

-   Formulate the waste load allocation problem and implement it in
    `JuMP`.
-   Draw a diagram showing the flows of waste between the cities and the
    facilities. Which facilities (if any) will not be used? Does this
    solution make sense?

In [36]:
#Define model and decision variables
waste_model = Model(HiGHS.Optimizer)

@variable(waste_model, W[i = 1:3, j = 1:3] >=0)
@variable(waste_model, R[i = 1:3, j = 1:3] >=0)
@variable(waste_model, Y[i = 1:3], Bin)

#Set objective
@objective(waste_model, Min, (2000*Y[1]) + (1500*Y[2]) + (2500*Y[3])
+ (57.5*W[1,1]) + (68*W[1,2]) + (82.5*W[1,3])
+ (72.5*W[2,1]) + (60.5*W[2,2]) + (75*W[2,3])
+ (69.5*W[3,1]) + (90.5*W[3,2]) + (90*W[3,3])
+ (98*R[2,1]) + (77*R[3,1]) + (82.5*R[2,3]))

#Set constraints

#City mass balance constraints
@constraint(waste_model,c1mb,  (W[1,1] + W[1,2]+ W[1,3]) == 100)
@constraint(waste_model,c2mb, (W[2,1] + W[2,2] + W[2,3]) == 90)
@constraint(waste_model,c3mb,  (W[3,1] + W[3,2] + W[3,3]) == 120)

#Residual mass balance constraints
@constraint(waste_model,WTEres,  R[3,1] == (0.16*(W[1,3] + W[2,3]+ W[3,3] + R[2,3])))
@constraint(waste_model,MRFres,  (R[2,3] + R[2,1]) == (0.6*(W[1,2] + W[2,2] + W[3,2])))

#Disposal limit constraints
@constraint(waste_model,WTEdis,  (W[1,3] + W[2,3] + W[3,3] + R[2,3]) <=210)
@constraint(waste_model,MRFdis,  (W[1,2] + W[2,2] + W[3,2]) <= 350)
@constraint(waste_model,LFdis,  (W[1,1] + W[2,1] + W[3,1] + R[3,1] + R[2,1]) <= 200)

#Commitment constraints
@constraint(waste_model,commit1,  Y[1] == 1)
@constraint(waste_model, commit2, (W[1,2] + W[2,2] + W[3,2]) <= 2000000*Y[2])
@constraint(waste_model, commit3, (W[1,3] + W[2,3] + W[3,3] + R[2,3]) <= (2000000* Y[3]))

#print(waste_model)
optimize!(waste_model)
@show value.(W)
@show value.(R)
@show value.(Y)

Running HiGHS 1.8.1 (git hash: 4a7f24ac6): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [2e-01, 2e+06]
  Cost   [6e+01, 2e+03]
  Bound  [1e+00, 1e+00]
  RHS    [1e+00, 4e+02]
Presolving model
9 rows, 14 cols, 37 nonzeros  0s
8 rows, 13 cols, 35 nonzeros  0s

Solving MIP model with:
   8 rows
   13 cols (2 binary, 0 integer, 0 implied int., 11 continuous)
   35 nonzeros
MIP-Timing:      0.0035 - starting analytic centre calculation

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

       

3-element Vector{Float64}:
  1.0
 -0.0
  1.0

## References

List any external references consulted, including classmates.